In [1]:
#!pip install selenium
#!pip install scrapy

In [2]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import random

In [3]:
driver = webdriver.Chrome(r'D:\UMD Master Year 2 College Work\Fall 2022-2023\INST737-IM01\chromedriver')
url = 'https://www.imdb.com/title/tt2488496/reviews?ref_=tt_ov_rt'
randomNumber = random.randint(0,4)
time.sleep(1)
driver.get(url)
time.sleep(1)
print(driver.title)
time.sleep(1)
body = driver.find_element(By.CSS_SELECTOR, 'body')
body.send_keys(Keys.END)
time.sleep(randomNumber)
# body.send_keys(Keys.PAGE_DOWN)
# time.sleep(1)
# body.send_keys(Keys.PAGE_DOWN)
# time.sleep(1)
# body.send_keys(Keys.PAGE_DOWN)

Star Wars: Episode VII - The Force Awakens (2015) - Star Wars: Episode VII - The Force Awakens (2015) - User Reviews - IMDb


In [4]:
sel = Selector(text = driver.page_source)
review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
more_review_pages = int(int(review_counts)/25)

In [5]:
for i in tqdm(range(more_review_pages)):
    try:
        css_selector = 'load-more-trigger'
        driver.find_element(By.ID, css_selector).click()
        body.send_keys(Keys.END)
        time.sleep(randomNumber)
    except:
        pass

100%|██████████| 202/202 [18:48<00:00,  5.59s/it]


In [6]:
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')
first_review = reviews[0]
sel2 = Selector(text = first_review.get_attribute('innerHTML'))
rating = sel2.css('.rating-other-user-rating span::text').extract_first().strip()

In [7]:
review = sel2.css('.text.show-more__control::text').extract_first().strip()
review_date = sel2.css('.review-date::text').extract_first().strip()
author = sel2.css('.display-name-link a::text').extract_first().strip()
review_title = sel2.css('a.title::text').extract_first().strip()
review_url = sel2.css('a.title::attr(href)').extract_first().strip()
#helpfulness = sel2.css('.actions.text-muted::text').extract_first().strip()
print('nRating:',rating)
print('nreview_title:',review_title)
print('nAuthor:',author)
print('nreview_date:',review_date)
print('nreview:',review)
#print('nhelpfulness:',helpfulness)

nRating: 1
nreview_title: Reboot, Sequel, Prequel, Re-Imagining Rebrand. Welcome to the Modern Hollywood.
nAuthor: Prichards12345
nreview_date: 19 January 2016
nreview: Star Wars movies used to be great. Remember that? Remember how they inspired a generation of loyal fans? Remember how they virtually saved cinema from terminal audience decline in the 70s? So scoot forward 40 years, and see the empty soulless rubbish lapped up by brainwashed Pavlovian audiences and critics. How much did Disney pay the critics for all those gushing reviews? Either that or the art of film criticism has been lost.


In [8]:
rating_list = []
review_date_list = []
review_title_list = []
author_list = []
review_list = []
review_url_list = []
error_url_list = []
error_msg_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

for d in tqdm(reviews):
    try:
        sel2 = Selector(text = d.get_attribute('innerHTML'))
        try:
            rating = sel2.css('.rating-other-user-rating span::text').extract_first()
        except:
            rating = np.NaN
        try:
            review = sel2.css('.text.show-more__control::text').extract_first()
        except:
            review = np.NaN
        try:
            review_date = sel2.css('.review-date::text').extract_first()
        except:
            review_date = np.NaN    
        try:
            author = sel2.css('.display-name-link a::text').extract_first()
        except:
            author = np.NaN    
        try:
            review_title = sel2.css('a.title::text').extract_first()
        except:
            review_title = np.NaN
        try:
            review_url = sel2.css('a.title::attr(href)').extract_first()
        except:
            review_url = np.NaN
        rating_list.append(rating)
        review_date_list.append(review_date)
        review_title_list.append(review_title)
        author_list.append(author)
        review_list.append(review)
        review_url_list.append(review_url)
    except Exception as e:
        error_url_list.append(url)
        error_msg_list.append(e)
review_df = pd.DataFrame({
    'Review_Date':review_date_list,
    'Author':author_list,
    'Rating':rating_list,
    'Review_Title':review_title_list,
    'Review':review_list,
    'Review_Url':review_url
    })

100%|██████████| 5062/5062 [03:14<00:00, 26.09it/s]


In [9]:
review_df

,Review_Date,Author,Rating,Review_Title,Review,Review_Url
0,19 January 2016,Prichards12345,1,"Reboot, Sequel, Prequel, Re-Imagining Rebrand...",Star Wars movies used to be great. Remember th...,/review/rw8471926/?ref_=tt_urv
1,14 January 2016,mcgrew,2,JJ does it again (and not in the good way)\n,"You'll notice that SW7 only has a ""based on ch...",/review/rw8471926/?ref_=tt_urv
2,7 April 2016,jaredpahl,1,Disney Sponsored Fan Fiction. The Force Awake...,"For those praising this film,",/review/rw8471926/?ref_=tt_urv
3,19 February 2016,Searsino,2,Star Wars: The Force is for Everyone Apparent...,Context: I grew up on these films. I didn't th...,/review/rw8471926/?ref_=tt_urv
4,4 April 2016,LouieInLove,3,Disney's Hype Wars Won. Star Wars Did Not - A...,Following the mass-marketing campaign extraord...,/review/rw8471926/?ref_=tt_urv
...,...,...,...,...,...,...
5057,17 February 2016,actionmaster,None,"Great, but not Fantastic\n",This is a Great movie and it will be the same ...,/review/rw8471926/?ref_=tt_urv
5058,14 December 2016,koriu,7,Good old Star Wars action but déjà vu\n,"First of all, as a huge Star Wars fan, I find ...",/review/rw8471926/?ref_=tt_urv
5059,27 October 2019,tommyhug,None,Darkness lies\n,"She lied. No wrist, only me trying leave her. ...",/review/rw8471926/?ref_=tt_urv
5060,3 August 2018,jaroschjr,8,The Prodigal Son Come Home at Last\n,"This movie is flawed, to say the least. Most o...",/review/rw8471926/?ref_=tt_urv


In [10]:
#5065 reviews

In [13]:
#review_df.to_csv(r'D:\UMD Master Year 2 College Work\Fall 2022-2023\INST737-IM01\INST737_Star_Wars_IMDB_StarWars7.csv')